# Agent 实战：智能路由、任务拆解和链路工程

> We always overestimate the change that will occur in the next two years and underestimate the change that will occur in the next ten. -- Bill Gates

大模型的输入是语言，输出也是语言，它终究是一种语言模态的东西。如果想要影响现实世界，比如执行程序、操纵机械臂，就需要一种通信协议，帮它完成自然语言到接口语言的转换，于是 **MCP** 出现了；囿于单个大模型的上下文窗口限制、参数量限制、思考时间限制，它无法完成复杂且长程的工作。如果有一个框架，把多个乃至多种大模型组织起来，通力协作，共同完成一项任务该有多好，于是 **Agent** 又出现了。是的，新技术出现得就是这么合理，接下来的事，大家都知道了。

- 2024 年 11 月，Anthropic 发布了 [MCP](https://github.com/modelcontextprotocol/servers)
- 2025 年 1 月，阿里发布 [Qwen Agent](https://github.com/QwenLM/Qwen-Agent)
- 2025 年 3 月，OpenAI 发布 [OpenAI Agents](https://github.com/openai/openai-agents-python)
- 2025 年 4 月，Google 发布 [A2A](https://github.com/google-a2a/A2A)

事情正在往想象中的方向发展 ......

## 一、引言

### 1. 尚未到来的涌现

把大象放进冰箱只需要三步。让大模型实现自举，也只需要三步：

1. 让大模型自己处理语料
2. 让大模型自己训练自己
3. 让大模型自己评估自己、反思、优化、调整超参、继续实验 ......

只要完成一次循环，即一次自我进化，理论上大模型就能完成两次、三次、乃至 N 次，这就是 **自举**。一旦大模型能自我演化，某种意义上，它就成了生物，也意味着技术奇点真正到来。至少目前，我们尚未触发这样的循环。可能是基座模型不够强大，也可能是 Agent 框架不够成熟，或者什么我们还不了解的原因。

就这个比较初级的 Agent，对智能的提升幅度来看，工程框架在 AGI 当中似乎不起决定性作用：并不是说，只要把大模型连起来，智能就会成倍地增加。要实现 AGI，恐怕还是要基座模型本身 AGI 才行。Agent 不是 Alpha，顶多算 Smart Beta.

2024 年，Agent 曾被寄予厚望。那一年技术大爆发。Sora, [GR](https://arxiv.org/abs/2402.17152), GPT-4o 等新技术层出不穷，大家对 AI 前景无限乐观。马斯克甚至说，到 2025 年，AI 可能比任何一个人都聪明 [[原文](https://x.com/elonmusk/status/1767738797276451090)]。事实是，2025 年最大的红利是 Agent，当红利释放后，想象中的智能涌现没有来。今年 AGI 的概率已经很小很小了。

### 2. 小米加步枪

Agent 的出发点是：模型能力不够，工程方法来凑。

一个大模型能力有限。通过组织多个模型一起工作，每个模型负责其中一块。然后像人一样组织它们，通过分发、交接、聚合等操作，构成一个个工作流。如此便能突破单个模型能力的上限，做更多、更复杂的事情。

```mermaid
graph LR
    A[感知] --> B[规划]
    B --> C[控制]
    C -.-> D[反思]
    D -.-> E[记忆]
```

Agent 工作时大多遵循以下模式：

- **感知**：一个任务过来，无论是 **被动接受** 任务信息，还是 **主动探索** 与任务有关的线索，这都属于“感知”。
- **规划**：有了对任务的感知，我们拆解任务，规划出一条或多条实现路径
- **控制**：有了实现路径，在路径上的每一个节点，通过 MCP 调用资源，执行任务；如果当前节点遇到困难，则动态地创建 sub-agent 继续分解任务
- **反思**：对各个路径的实现结果进行评估。如果某条路径失败，总结失败经验，重新来过，直到抵达重试次数上限
- **记忆**：收集本次任务信息，提炼重点，存入知识库。下次执行相似任务时，从知识库中取出，作为先验知识

一个大模型搞不定，就用多个大模型解决；没有大炮没关系，就用小米加步枪战斗。

### 3. 还有 MCP

如果说 Agent 起着调度作用，像人的大脑，那么 MCP 就像人的手。

MCP 相当于引入了 `Action -> React` Pair。大模型可以通过 MCP 与现实世界交互，获得即时反馈。它可以交互的东西很多，比如：

- **FastAPI**（接入中台能力）
- **PostgreSQL**（长期记忆）
- **Redis**（短期记忆）
- **Docker**（代码执行能力）

Agent 其实兼具眼和脑的功能，加上 MCP 这个手，事情变得有趣起来。毕竟人类也是靠 **眼 -> 脑 -> 手** 循环，把活干起来的。

### 4. Agent 能做什么

> 回到务实的问题。

<!-- Agent 是大模型的调度器，它擅长的无非是大模型擅长的那些，外加一些由组织和规划产生的能力。 -->

Agent 的能力，是组织规划的能力。

有些公司，把 Agent 当作中台服务的调度器。用 MCP 把中台服务轮询一遍，再用大模型把轮询的结果总结一下。这当然也是一种用法，但有点画蛇添足。因为用传统的 `for loop` 和 `if else` 控制块，也能实现相同的逻辑。

**Agent 的价值在传统编程的能力圈之外。**

新技术落地是这样的，总会有拿着锤子找钉子的过程，但终究要做传统编程做不到的事情。

**1）智能路由 (Function Calling)**

我们确实可以把 Agent 当调度器使用，但不是简单 `if else` 可以实现的那种。

```mermaid
graph LR
    A[任务] --> B[智能路由]
    B --> C[mcp1]
    B --> D[mcp2]
    B --> E[mcp3]
```

> **Note:** 举个例子，我们写一个 Agent 来 **核查客诉是否属实**。如果用户投诉“商家未发货”，就查发货记录；如果投诉“好几天了，还没收到货”，就去查物流表；如果投诉“商家真没素质”，就去查聊天记录。传统编程处理这种问题，遇到长尾 case 会没法解，需要人工介入。这种本身属于语言模态，问题又无法特别明确定义的，就适合用 **智能路由** 的方式解决。

**2）任务拆解（ReActChat）**

理想情况下，我们把 MCP 开发好丢给 Agent，告诉它我们想要什么。Agent 应该自己将任务拆解成若干步骤，再将每个步骤交由 MCP 处理。遇到错误会自己重试。这种功能已经被开发出来了，它叫 **ReAct 模式**。

ReAct (Reasoning-Action) 是一种结合了 **思考决策 (Reasoning)** 和 **环境交互 (Action)** 的智能体模式。其核心逻辑是：先规划一系列操作，依次执行，再基于操作结果进一步思考，循环往复直至问题解决。ReAct 模式目前不太好用，即使上了 `Qwen3` 和 `Deepseek-0528` 最大尺寸的模型，只要问题一难，就会有相当程度的失败概率。我们遇到失败，就往里掺链路工程，一点点地把良率做上去。因此和同事戏言：有多少人工，就有多少人工智能。

```mermaid
graph LR
A[收到任务] --> B[任务拆解]
B --> C[步骤1]
C --> D[步骤2]
D --> E[步骤n]
E --> F{结果足够吗?}
F -->|是| G[整理结果并回答]
F -->|否| H[反思历史，调整思路]
H --> B
```

**3）深度研究（Deep Research）**

当你需要解决业务难题时，仅凭 Function Calling 或 ReActChat 是不够的，你要做 Agent 的 **链路工程**。具体来说，你要将 Function Calling 和 ReAct 作为基础组件，开发许多工作流。

```mermaid
graph LR
    A[监控] -->|发现盗刷| B[风控引擎]
    B --> C[支付基础风险]
    B --> D[设备基础风险]
    B --> E[登录基础风险]
    B --> F[黑/灰名单库]

    subgraph 基础风险模块
    C[支付基础风险] --> G[风险判别]
    D --> G
    E --> G
    F --> G
    end

    G -->|研判信息不足触发| H[动态调度]

    subgraph 高计算模块
    H -.-> I[k 度风险节点] --> N[风险报告]
    H -.-> J[FPGrowth] --> N
    H -.-> K[SynchroTrap] --> N
    H -.-> L[FastUnfolding] --> N
    H -.-> M[LightGBM] --> N
    end

    N -->|黑标扩散| O[周边风险]

    subgraph 扩召模块
    O -.-> P[邻域召回路]
    O -.-> Q[Graph Embedding 召回路]
    O -.-> R[时序相似召回路]
    end

    P --> S[团伙打标]
    Q --> S
    R --> S
```

以上是一个 Agentic 风格的 **风控引擎**。当监控发现用户在支付路由下有风险时，监控立即向风控引擎发送一条消息，形如：

```json
{
    "uid": 19335701,
    "route": "pay",
    "reason": "card_fraud",
    "timestamp": "2025-06-01 19:21:09"
}
```

风控引擎 Agent 会运行一个初步的排查程序，生成简单的风险报告。如果发现，简单的报告不足以揭示风险，则调用更高消耗的计算模块，获得更多风险信息。最后它综合所有信息进行研判，告诉我们这个用户是不是坏人。如果用户是坏人，它还能继续探查坏人的周边用户，召回更多坏人，进而打击整个黑产团伙。

参考：[deep-research](https://openai.com/index/introducing-deep-research/)

### 5. 世之显学

Agent 之后，**Agentic** 已经成为一种显学，万物皆可 Agentic，比如 RAG 现在叫 [Agentic RAG](https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_agentic_rag/)。这是好事，说明 Agent 真的有用，它可以为很多业务带来提升，比如智能客服、内容风控、商业智能 (ChatBI) 等等等等

蛮不可思议的。试想短短数年间，我们经历了：

1. 互联网+
2. AI+
3. LLM+
4. Agent+

每项技术都发展得很缓慢，但是跳出来一看，就被技术大爆发所震惊。不知道的还以为我们 [2017 年](https://arxiv.org/abs/1706.03762) 捡到外星飞船了呢。